In [1]:
## this notebook aims to prepare EARNINGS CALL files for transcripton. that includes
## - pre process to correct file formats/encoding 
## - take metadata about the audio
## - submit a job for transcribing 

In [ ]:
# read audio file (full) -> upload from user or LINK 
# input meta for the file 
# transcribe 




In [ ]:
# upload to s3 -> for dev purpose 'data' folder is s3 
# pull audio -> create meta -> start processing


## meta 
- filename 
- type 
- format 
- url 
- length of audio 
- org name 
- about org/convo [optional]
- number of speakers [opt]
- 

In [ ]:
# note 
# transcript will be saved in the database as 'raw'.  post verification by user it is stored as final trx.
# embedding happens post user validation 
# embedding strategy is default - tbd 
## default - full file, chunk 

In [ ]:
from enum import Enum 
from enums import tsFormats
class RecordingTypes(Enum):
    MP3 = 'mp3'
    WAV = 'wav'
    M4A = 'm4a'

class RecordingTypes(Enum):
   CONCALL = 'concall'
   INTERVIEW = 'interview'
   

class AudioFileMeta:
    def __init__(self, 
                 file_name:str, 
                 input_file_format:RecordingTypes,
                 recording_type:RecordingTypes,
                 audio_length:float,
                 ) -> None:
        
        pass

class TranscriptMetadata:
    def __init__(self, 
                 audio_file_name:str, 
                 transcript_format:tsFormats,
                    
                 ) -> None:
        
        pass